# Analyse X_train_preprocessed

Notebook d'analyse des données preprocessées pour valider les transformations appliquées.

In [68]:
import polars as pl
import numpy as np
from pathlib import Path
import glob

## 1. Chargement des données

In [69]:
# Trouver le dernier fichier preprocessed
data_path = Path('../../data')
preprocessed_files = sorted(data_path.glob('X_train_preprocessed_*.csv'))

if not preprocessed_files:
    raise FileNotFoundError("Aucun fichier X_train_preprocessed_*.csv trouvé")

latest_file = preprocessed_files[-1]
print(f"Fichier chargé: {latest_file.name}")

# Charger les données
df_preprocessed = pl.read_csv(latest_file)
df_original = pl.read_csv(data_path / 'X_train.csv')

Fichier chargé: X_train_preprocessed_20251129_180836.csv


## 2. Statistiques générales

In [70]:
print("="*80)
print("STATISTIQUES GÉNÉRALES")
print("="*80)

print("\n📊 DIMENSIONS")
print(f"  Données originales:     {df_original.shape}")
print(f"  Données preprocessées:  {df_preprocessed.shape}")
print(f"  Réduction colonnes:     {df_original.shape[1] - df_preprocessed.shape[1]} ({(df_original.shape[1] - df_preprocessed.shape[1])/df_original.shape[1]*100:.1f}%)")

print("\n📉 VALEURS MANQUANTES")
null_count_original = df_original.null_count().sum_horizontal()[0]
null_count_preprocessed = df_preprocessed.null_count().sum_horizontal()[0]
total_cells_original = df_original.shape[0] * df_original.shape[1]
total_cells_preprocessed = df_preprocessed.shape[0] * df_preprocessed.shape[1]

print(f"  Original:      {null_count_original:,} ({null_count_original/total_cells_original*100:.2f}%)")
print(f"  Preprocessé:   {null_count_preprocessed:,} ({null_count_preprocessed/total_cells_preprocessed*100:.2f}%)")
print(f"  Amélioration:  {null_count_original - null_count_preprocessed:,} valeurs manquantes en moins")

STATISTIQUES GÉNÉRALES

📊 DIMENSIONS
  Données originales:     (1172086, 307)
  Données preprocessées:  (1172086, 110)
  Réduction colonnes:     197 (64.2%)

📉 VALEURS MANQUANTES
  Original:      213,250,711 (59.26%)
  Preprocessé:   0 (0.00%)
  Amélioration:  213,250,711 valeurs manquantes en moins


In [71]:
# Afficher aperçu
print("\n📋 APERÇU DES DONNÉES PREPROCESSÉES (5 premières lignes)")
df_preprocessed.head(5)


📋 APERÇU DES DONNÉES PREPROCESSÉES (5 premières lignes)


,OECD,ADMINMODE,LANGTEST_COG,ST003D02T,ST004D01T,EFFORT1,EFFORT2,GRADE,IMMIG,IC171,ST038,ST034,PA042,IC173,WB154,IC174,ST021,PA183,WB163,PA166,ST250,ST296,ST353,ST349,WB164,ST062,ST254,IC170,AGE,COBN_S,WB165,FL169,WB166,ST355,IC177,ST345,…,reading_q1_total_timing,reading_q2_total_timing,reading_q3_total_timing,reading_q4_total_timing,reading_q5_total_timing,reading_q6_total_timing,reading_q7_total_timing,reading_q8_total_timing,math_q1_total_timing,math_q2_total_timing,math_q3_total_timing,math_q4_total_timing,math_q5_total_timing,math_q6_total_timing,math_q7_total_timing,math_q8_total_timing,math_q9_total_timing,math_q10_total_timing,math_q11_total_timing,math_q12_total_timing,science_q1_total_timing,science_q2_total_timing,science_q3_total_timing,science_q4_total_timing,science_q5_total_timing,science_q6_total_timing,science_q7_total_timing,science_q8_total_timing,science_q9_total_timing,science_q10_total_timing,Year,CNTRYID,CNTSCHID,CNTSTUID,Score_Support_Enseignant,OCOD1_grouped,OCOD2_grouped
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64
384002,1.0,2.0,322.0,2.0,1.0,8.0,8.0,0.0,2.0,0.0,0.0,2.333333,0.0,0.0,2.0,0.0,0.0,0.0,6.5,0.0,4.0,1.0,0.0,0.0,0.0,0.333333,3.333333,0.0,15.75,52800.0,1.0,0.0,1.0,0.0,0.0,2.0,…,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,18.736,23.05525,162.72,110.641333,96.368333,71.8135,98.875,63.6894,37.232,88.274667,-1.0,-1.0,84.299333,90.156,66.612,73.5476,68.912,56.62325,104.1905,132.62325,105.785667,-1.0,2022.0,528.0,5.2800132e7,5.2801144e7,1.666667,3,2
1118072,0.0,2.0,803.0,11.0,2.0,7.0,6.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.42,3100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,…,70.44475,10.03575,4.8738,67.025602,78.74843,8.368284,112.711,69.975711,50.234,80.788,48.059,116.718,61.003,34.117,56.901668,76.615,21.500334,27.087,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2018.0,31.0,3.100106e6,3.100424e6,1.666667,2,9
845454,1.0,2.0,493.0,5.0,1.0,8.0,10.0,0.0,1.0,3.5,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,1.166667,0.0,3.25,0.0,0.0,8.0,1.5,16.0,25000.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,…,110.500602,64.958,35.507602,63.326145,4.517952,39.792332,92.942984,82.727664,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,81.443664,190.243406,36.257,148.0735,2.12125,90.540336,57.954,84.633,45.332,87.6865,2018.0,250.0,2.500001e7,2.5005207e7,1.333333,3,3
1728613,2.0,2.0,156.0,3.0,2.0,8.0,10.0,0.0,1.0,1.6,0.0,2.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.6,0.0,0.75,0.0,0.333333,2.0,1.4,16.17,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,…,79.73975,68.604664,53.5655,61.712332,63.6005,114.817336,81.293336,59.014332,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,62.224668,83.289664,91.51075,100.6495,82.3785,74.34825,91.230664,49.789,37.855,-1.0,2015.0,971.0,9.710024e7,9.7127584e7,0.666667,-1,-1
1083243,0.0,2.0,313.0,9.0,1.0,10.0,10.0,-1.0,1.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,15.5,60800.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,…,65.393,57.448,75.7315,126.398602,54.334801,4.223181,106.763336,41.3245,137.692,36.046,43.09575,82.2825,121.0105,191.303,77.024336,197.619672,64.659668,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2018.0,608.0,6.0800071e7,6.0802698e7,4.0,9,7


In [72]:
# Info sur les colonnes
print("\n📋 TYPES DE DONNÉES")
dtypes = df_preprocessed.schema
dtype_counts = {}
for dtype in dtypes.values():
    dtype_str = str(dtype)
    dtype_counts[dtype_str] = dtype_counts.get(dtype_str, 0) + 1

for dtype, count in sorted(dtype_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {dtype}: {count} colonnes")


📋 TYPES DE DONNÉES
  Float64: 107 colonnes
  Int64: 3 colonnes


In [73]:
[col for col in df_preprocessed.columns if df_preprocessed[col].dtype == pl.String]

[]

## 3. Vérification des transformations

### 3.1 Variables supprimées

In [74]:
# Variables redondantes ordinales qui devaient être supprimées
redundant_ordinal = ['ST005', 'ST007', 'ST253', 'ST255', 'ST097']

# Variables métadonnées catégorielles qui devaient être supprimées
metadata_categorical = [
    'Option_CT', 'Option_FL', 'Option_ICTQ', 'Option_PQ',
    'Option_TQ', 'Option_UH', 'Option_WBQ',
    'CYC', 'NatCen', 'SUBNATIO'
]

# Variables redondantes catégorielles qui devaient être supprimées
redundant_categorical = [
    'LANGTEST_PAQ', 'LANGTEST_QQQ', 'ST003D03T', 'ST001D01T',
    'PA008', 'PA162', 'OCOD3'
]

print("="*80)
print("VÉRIFICATION DES SUPPRESSIONS")
print("="*80)

print("\n🔢 Variables ordinales redondantes (devaient être supprimées):")
for var in redundant_ordinal:
    if var in df_original.columns:
        status = "❌ PRÉSENTE" if var in df_preprocessed.columns else "✅ SUPPRIMÉE"
        print(f"  {var}: {status}")
    else:
        print(f"  {var}: ⚠️  Absente du dataset original")

print("\n🏷️  Variables métadonnées catégorielles (devaient être supprimées):")
for var in metadata_categorical:
    if var in df_original.columns:
        status = "❌ PRÉSENTE" if var in df_preprocessed.columns else "✅ SUPPRIMÉE"
        print(f"  {var}: {status}")
    else:
        print(f"  {var}: ⚠️  Absente du dataset original")

print("\n🏷️  Variables redondantes catégorielles (devaient être supprimées):")
for var in redundant_categorical:
    if var in df_original.columns:
        status = "❌ PRÉSENTE" if var in df_preprocessed.columns else "✅ SUPPRIMÉE"
        print(f"  {var}: {status}")
    else:
        print(f"  {var}: ⚠️  Absente du dataset original")

VÉRIFICATION DES SUPPRESSIONS

🔢 Variables ordinales redondantes (devaient être supprimées):
  ST005: ✅ SUPPRIMÉE
  ST007: ✅ SUPPRIMÉE
  ST253: ✅ SUPPRIMÉE
  ST255: ✅ SUPPRIMÉE
  ST097: ✅ SUPPRIMÉE

🏷️  Variables métadonnées catégorielles (devaient être supprimées):
  Option_CT: ✅ SUPPRIMÉE
  Option_FL: ✅ SUPPRIMÉE
  Option_ICTQ: ✅ SUPPRIMÉE
  Option_PQ: ✅ SUPPRIMÉE
  Option_TQ: ✅ SUPPRIMÉE
  Option_UH: ✅ SUPPRIMÉE
  Option_WBQ: ✅ SUPPRIMÉE
  CYC: ✅ SUPPRIMÉE
  NatCen: ✅ SUPPRIMÉE
  SUBNATIO: ✅ SUPPRIMÉE

🏷️  Variables redondantes catégorielles (devaient être supprimées):
  LANGTEST_PAQ: ✅ SUPPRIMÉE
  LANGTEST_QQQ: ✅ SUPPRIMÉE
  ST003D03T: ✅ SUPPRIMÉE
  ST001D01T: ✅ SUPPRIMÉE
  PA008: ✅ SUPPRIMÉE
  PA162: ✅ SUPPRIMÉE
  OCOD3: ✅ SUPPRIMÉE


### 3.2 Scores composites

In [75]:
print("\n" + "="*80)
print("VÉRIFICATION DES SCORES COMPOSITES")
print("="*80)

composite_scores = [
    ('Score_Support_Parental', ['PA003', 'ST300']),
    ('Score_Support_Enseignant', ['ST100', 'ST270'])
]

for score_name, source_vars in composite_scores:
    print(f"\n📊 {score_name}")
    
    # Vérifier présence des variables sources dans original
    sources_in_original = [v for v in source_vars if v in df_original.columns]
    print(f"  Variables sources dans original: {sources_in_original}")
    
    if score_name in df_preprocessed.columns:
        print(f"  ✅ Score créé")
        # Vérifier que les variables sources ont été supprimées
        sources_in_preprocessed = [v for v in source_vars if v in df_preprocessed.columns]
        if sources_in_preprocessed:
            print(f"  ⚠️  Variables sources encore présentes: {sources_in_preprocessed}")
        else:
            print(f"  ✅ Variables sources supprimées")
    else:
        if len(sources_in_original) >= 2:
            print(f"  ⚠️  Score NON créé (sources disponibles)")
        else:
            print(f"  ℹ️  Score NON créé (sources manquantes dans original)")


VÉRIFICATION DES SCORES COMPOSITES

📊 Score_Support_Parental
  Variables sources dans original: ['PA003', 'ST300']
  ⚠️  Score NON créé (sources disponibles)

📊 Score_Support_Enseignant
  Variables sources dans original: ['ST100', 'ST270']
  ✅ Score créé
  ✅ Variables sources supprimées


### 3.3 Regroupement ISCO

In [76]:
print("\n" + "="*80)
print("VÉRIFICATION DES REGROUPEMENTS ISCO")
print("="*80)

isco_vars = ['OCOD1', 'OCOD2']

for var in isco_vars:
    grouped_var = f"{var}_grouped"
    
    print(f"\n📊 {var}")
    
    if var in df_original.columns:
        original_unique = df_original.select(pl.col(var).n_unique()).item()
        print(f"  Cardinalité originale: {original_unique}")
        
        if var in df_preprocessed.columns:
            print(f"  ❌ Variable originale encore présente")
        else:
            print(f"  ✅ Variable originale supprimée")
        
        if grouped_var in df_preprocessed.columns:
            grouped_unique = df_preprocessed.select(pl.col(grouped_var).n_unique()).item()
            print(f"  ✅ Variable groupée créée")
            print(f"  Cardinalité groupée: {grouped_unique}")
            print(f"  Réduction: {original_unique} → {grouped_unique} (-{(original_unique-grouped_unique)/original_unique*100:.1f}%)")
            
            # Afficher la distribution
            print(f"  Distribution:")
            distribution = df_preprocessed.group_by(grouped_var).agg(pl.count()).sort(grouped_var)
            print(distribution)
        else:
            print(f"  ❌ Variable groupée NON créée")
    else:
        print(f"  ⚠️  Absente du dataset original")


VÉRIFICATION DES REGROUPEMENTS ISCO

📊 OCOD1
  Cardinalité originale: 774
  ✅ Variable originale supprimée
  ✅ Variable groupée créée
  Cardinalité groupée: 11
  Réduction: 774 → 11 (-98.6%)
  Distribution:
shape: (11, 2)
┌───────────────┬────────┐
│ OCOD1_grouped ┆ count  │
│ ---           ┆ ---    │
│ i64           ┆ u32    │
╞═══════════════╪════════╡
│ -1            ┆ 348119 │
│ 0             ┆ 292    │
│ 1             ┆ 43904  │
│ 2             ┆ 167601 │
│ 3             ┆ 64873  │
│ …             ┆ …      │
│ 5             ┆ 132603 │
│ 6             ┆ 12139  │
│ 7             ┆ 25148  │
│ 8             ┆ 9340   │
│ 9             ┆ 321833 │
└───────────────┴────────┘

📊 OCOD2
  Cardinalité originale: 775
  ✅ Variable originale supprimée
  ✅ Variable groupée créée
  Cardinalité groupée: 11
  Réduction: 775 → 11 (-98.6%)
  Distribution:
shape: (11, 2)
┌───────────────┬────────┐
│ OCOD2_grouped ┆ count  │
│ ---           ┆ ---    │
│ i64           ┆ u32    │
╞═══════════════╪═══════

/var/folders/cn/f1c072zn49n5lbvp5frkrh500000gn/T/ipykernel_59414/1363085826.py:29: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  distribution = df_preprocessed.group_by(grouped_var).agg(pl.count()).sort(grouped_var)


## 4. Statistiques descriptives

In [77]:
print("\n" + "="*80)
print("STATISTIQUES DESCRIPTIVES DES COLONNES")
print("="*80)

# Colonnes numériques
numeric_cols = [col for col in df_preprocessed.columns if df_preprocessed[col].dtype in [pl.Int64, pl.Int32, pl.Float64, pl.Float32]]

if numeric_cols:
    print(f"\n📊 Variables numériques ({len(numeric_cols)} colonnes)")
    print(df_preprocessed.select(numeric_cols).describe())
else:
    print("\n⚠️  Aucune variable numérique détectée")


STATISTIQUES DESCRIPTIVES DES COLONNES

📊 Variables numériques (110 colonnes)
shape: (9, 111)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆           ┆ OECD      ┆ ADMINMODE ┆ … ┆ CNTSTUID  ┆ Score_Sup ┆ OCOD1_gro ┆ OCOD2_gr │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ port_Ense ┆ uped      ┆ ouped    │
│ str       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ ignant    ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆ ---       ┆ f64       ┆ f64      │
│           ┆           ┆           ┆           ┆   ┆           ┆ f64       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 1.172086e ┆ 1.172086e ┆ 1.172086e ┆ … ┆ 1.172086e ┆ 1.172086e ┆ 1.172086e ┆ 1.172086 │
│           ┆ 6         ┆ 6         ┆ 6         ┆   ┆ 6         ┆ 6         ┆ 6         ┆ e6     

In [78]:
# Distribution des valeurs manquantes par colonne
print("\n📉 TOP 50 COLONNES AVEC LE PLUS DE VALEURS MANQUANTES")

missing_by_col = []
for col in df_preprocessed.columns:
    null_count = df_preprocessed[col].null_count()
    null_pct = (null_count / len(df_preprocessed)) * 100
    missing_by_col.append({'column': col, 'null_count': null_count, 'null_pct': null_pct})

missing_df = pl.DataFrame(missing_by_col).sort('null_count', descending=True).head(30)
print(missing_df)


📉 TOP 50 COLONNES AVEC LE PLUS DE VALEURS MANQUANTES
shape: (30, 3)
┌──────────────┬────────────┬──────────┐
│ column       ┆ null_count ┆ null_pct │
│ ---          ┆ ---        ┆ ---      │
│ str          ┆ i64        ┆ f64      │
╞══════════════╪════════════╪══════════╡
│              ┆ 0          ┆ 0.0      │
│ OECD         ┆ 0          ┆ 0.0      │
│ ADMINMODE    ┆ 0          ┆ 0.0      │
│ LANGTEST_COG ┆ 0          ┆ 0.0      │
│ ST003D02T    ┆ 0          ┆ 0.0      │
│ …            ┆ …          ┆ …        │
│ WB164        ┆ 0          ┆ 0.0      │
│ ST062        ┆ 0          ┆ 0.0      │
│ ST254        ┆ 0          ┆ 0.0      │
│ IC170        ┆ 0          ┆ 0.0      │
│ AGE          ┆ 0          ┆ 0.0      │
└──────────────┴────────────┴──────────┘


Il y a 28 colonnes où il reste des NaNs.

In [79]:
[col for col in df_preprocessed.columns if df_preprocessed[col].null_count() > 0]

[]

## 5. Résumé des validations

In [80]:
print("\n" + "="*80)
print("RÉSUMÉ DES VALIDATIONS")
print("="*80)

validations = []

# Check suppressions ordinales
ordinal_removed = sum(1 for v in redundant_ordinal if v in df_original.columns and v not in df_preprocessed.columns)
ordinal_expected = sum(1 for v in redundant_ordinal if v in df_original.columns)
validations.append(('Variables ordinales supprimées', ordinal_removed, ordinal_expected))

# Check suppressions métadonnées
metadata_removed = sum(1 for v in metadata_categorical if v in df_original.columns and v not in df_preprocessed.columns)
metadata_expected = sum(1 for v in metadata_categorical if v in df_original.columns)
validations.append(('Métadonnées supprimées', metadata_removed, metadata_expected))

# Check suppressions redondances catégorielles
redundant_removed = sum(1 for v in redundant_categorical if v in df_original.columns and v not in df_preprocessed.columns)
redundant_expected = sum(1 for v in redundant_categorical if v in df_original.columns)
validations.append(('Redondances catégorielles supprimées', redundant_removed, redundant_expected))

# Check regroupements ISCO
isco_grouped = sum(1 for v in isco_vars if f"{v}_grouped" in df_preprocessed.columns)
isco_expected = sum(1 for v in isco_vars if v in df_original.columns)
validations.append(('Regroupements ISCO créés', isco_grouped, isco_expected))

print("\n")
for name, actual, expected in validations:
    status = "✅" if actual == expected else "⚠️"
    print(f"{status} {name}: {actual}/{expected}")

print("\n" + "="*80)


RÉSUMÉ DES VALIDATIONS


✅ Variables ordinales supprimées: 5/5
✅ Métadonnées supprimées: 10/10
✅ Redondances catégorielles supprimées: 7/7
✅ Regroupements ISCO créés: 2/2

